# P4Lang Tutorials of FABRIC

This notebook walks the user through setting up a FABRIC eperiment that is suitiable for completing the P4 tutorials created by [P4Lang](https://github.com/p4lang/tutorials). The tutorials were origianlly designed to use a mininet topology. This example replaces the mininet topology with a FABRIC experiemnt topology that may span multiple sites across the FABRIC testbed.

Additional resources:
- [FABRIC Knowledge Base](https://learn.fabric-testbed.net/)
- [FABRIC Forums](https://learn.fabric-testbed.net/forums/)
- [P4Lang Tutorials](https://github.com/p4lang/tutorials)
- [P4Lang YouTube Presentations](https://www.youtube.com/channel/UCOQAFkDKucJWr-KafdJsdIQ)

## Basic FABRIC Slice Configuration

In [13]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

In [14]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Exception: b'{\n    "errors": [\n        {\n            "details": "(invalid_grant) invalid refresh token",\n            "message": "Internal Server Error"\n        }\n    ],\n    "size": 1,\n    "status": 500,\n    "type": "error"\n}'


## Configure Slice Parameters

This section builds the experiment slice 

<img src="figs/fabric_slice.png" width="800"/>



In [3]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# Slice 
slice_name = 'P4_VCC_hm'

# [site1,site2,site3] = fablib.get_random_sites(count=3)
site1 = site2 = site3 = 'MAX'
print(f"Sites: {site1},{site2},{site3}")


# Switches
s1_name = "s1"
s2_name = "s2"
s3_name = "s3"

switch_cores = 2
switch_ram = 8
switch_disk = 100

# Hosts
h1_name = "h1"
h11_name = "h11"
h2_name = "h2"
h22_name = "h22"


h1_subnet=IPv4Network('10.0.1.0/24')
h1_addr=IPv4Address('10.0.1.1')
h11_addr=IPv4Address('10.0.1.11')

h2_subnet=IPv4Network('10.0.2.0/24')
h2_addr=IPv4Address('10.0.2.2')
h22_addr=IPv4Address('10.0.2.22')

host_cores = 2
host_ram = 8
host_disk = 20

net_h1_name = 'net_h1'
net_h11_name = 'net_h11'
net_h2_name = 'net_h2'
net_h22_name = 'net_h22'

net_s1_s2_name = 'net_s1_s2'
net_s2_s3_name = 'net_s2_s3'
net_s1_s3_name = 'net_s1_s3'

s1_iface_h1,s1_iface_h11,s1_iface_to_s2,s1_iface_to_s3 = "s1_iface_h1","s1_iface_h11","s1_iface_to_s2","s1_iface_to_s3"  
s2_iface_h2,s2_iface_h22,s2_iface_to_s1,s2_iface_to_s3 = "s2_iface_h2","s2_iface_h22","s2_iface_to_s1","s2_iface_to_s3" 
s3_iface_to_s1,s3_iface_to_s2 = "s3_iface_to_s1","s3_iface_to_s2"

# All node properties
image = 'default_ubuntu_20'

Sites: MAX,MAX,MAX


### Create the Slice

In [4]:
import datetime
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    def addSwitchNIC(nodeName,site,ifaceNames):
        node = slice.add_node(name=nodeName, site=site,  image=image, cores=switch_cores, ram=switch_ram, disk=switch_disk)
        ifaceObj = {}
        for name in ifaceNames:
            iface =  node.add_component(model='NIC_Basic', name=name).get_interfaces()[0]
            ifaceObj[name] = iface
        return node, ifaceObj
    
    # Add switch node s1
    s1, s1_ifaces = addSwitchNIC(s1_name,site1,[s1_iface_h1,s1_iface_h11,s1_iface_to_s2,s1_iface_to_s3])

    # Add switch node s2
    s2, s2_ifaces = addSwitchNIC(s2_name,site2,[s2_iface_h2,s2_iface_h22,s2_iface_to_s1,s2_iface_to_s3])
    
    # Add switch node s3
    s3, s3_ifaces = addSwitchNIC(s3_name,site3,[s3_iface_to_s1,s3_iface_to_s2])
    
    def addHostNIC(nodeName,componentName,site):
        node = slice.add_node(name=nodeName, site=site, image=image, cores=host_cores, ram=host_ram, disk=host_disk)
        iface = node.add_component(model='NIC_Basic', name=componentName).get_interfaces()[0]
        return node,iface
    
    # Add host node h1
    h1,h1_iface = addHostNIC(h1_name,"h1_nic",site1)

    # Add host node h11
    h11 , h11_iface = addHostNIC(h11_name,"h11_nic",site1)
    
    # Add host node h2
    h2, h2_iface = addHostNIC(h2_name,"h2_nic",site2)
    
    # Add host node h3
    h22, h22_iface = addHostNIC(h22_name,"h22_nic",site2)
    
    #Add swtich networks
    switch_net1 = slice.add_l2network(name=net_s1_s2_name, interfaces=[s1_ifaces[s1_iface_to_s2], s2_ifaces[s2_iface_to_s1]])
    swtich_net2 = slice.add_l2network(name=net_s2_s3_name, interfaces=[s2_ifaces[s2_iface_to_s3], s3_ifaces[s3_iface_to_s2]])
    swtich_net3 = slice.add_l2network(name=net_s1_s3_name, interfaces=[s3_ifaces[s3_iface_to_s1], s1_ifaces[s1_iface_to_s3]])

    #Add host networks 
    host_net1 = slice.add_l2network(name=net_h1_name, interfaces=[s1_ifaces[s1_iface_h1], h1_iface])
    host_net11 = slice.add_l2network(name=net_h11_name, interfaces=[s1_ifaces[s1_iface_h11], h11_iface])
    host_net2 = slice.add_l2network(name=net_h2_name, interfaces=[s2_ifaces[s2_iface_h2], h2_iface])
    host_net22 = slice.add_l2network(name=net_h22_name, interfaces=[s2_ifaces[s2_iface_h22], h22_iface])
    
    #Submit Slice Request
    slice.submit() 
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    


-----------  ------------------------------------
Slice Name   P4_VCC_hm
Slice ID     50edee1e-30fb-436e-af84-2528429d6031
Slice State  StableOK
Lease End    2022-09-02 23:56:09 +0000
-----------  ------------------------------------

Retry: 17, Time: 257 sec

ID                                    Name    Site    Host                         Cores    RAM    Disk  Image              Management IP    State    Error
------------------------------------  ------  ------  -------------------------  -------  -----  ------  -----------------  ---------------  -------  -------
cd3758aa-e9b1-401b-b580-5f9526ba3811  s1      MAX     max-w5.fabric-testbed.net        2      8     100  default_ubuntu_20  63.239.135.68    Active
879b5a27-b90d-4cd9-a8b4-e16f7e9adbdc  s2      MAX     max-w5.fabric-testbed.net        2      8     100  default_ubuntu_20  63.239.135.115   Active
ec5bc966-6e5e-4e62-80ee-38dab1eefe2c  s3      MAX     max-w5.fabric-testbed.net        2      8     100  default_ubuntu_20  63.2

In [5]:
def checkInetConnection(node):
    try:

        #If the node is an IPv6 Node then configure NAT64
        if type(ip_address(node.get_management_ip())) is IPv6Address:
            print("This code only works with ip4, reconfigure nodes")
            return

        #Access non-IPv6 Services
        stdout, stderr = node.execute(f'sudo apt-get update -qq && sudo apt install net-tools && git clone https://github.com/fabric-testbed/jupyter-examples.git')
        # print(stdout)
        # print(stderr)

        stdout, stderr = node.execute(f'ls jupyter-examples')
        print(stdout)
        print(stderr)
        
        stdout, stderr = node.execute(f'ifconfig | grep ens')
        print(stdout)
        print(stderr)


    except Exception as e:
        print(f"Exception: {e}")

In [6]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        checkInetConnection(node)
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Node:
   Name              : s1
LICENSE
Readme.md
fabric_examples
start_here.ipynb


ens3: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 9000
ens7: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
ens8: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
ens9: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
ens10: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500


Node:
   Name              : s2
LICENSE
Readme.md
fabric_examples
start_here.ipynb


ens3: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 9000
ens7: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
ens8: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
ens9: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
ens10: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500


Node:
   Name              : s3
LICENSE
Readme.md
fabric_examples
start_here.ipynb


ens3: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 9000
ens7: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
ens8: flags=4163<UP,BROADCAST,R

### Print Details

In [7]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print("----")
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        # print(f"   Cores             : {node.get_cores()}")
        # print(f"   RAM               : {node.get_ram()}")
        # print(f"   Disk              : {node.get_disk()}")
        # print(f"   Image             : {node.get_image()}")
        # print(f"   Image Type        : {node.get_image_type()}")
        # print(f"   Host              : {node.get_host()}")
        # print(f"   Site              : {node.get_site()}")
        # print(f"   Management IP     : {node.get_management_ip()}")
        # print(f"   Reservation ID    : {node.get_reservation_id()}")
        # print(f"   Reservation State : {node.get_reservation_state()}")
        ssh = node.get_ssh_command().replace("ssh -i /home/fabric/work/fabric_config/.ssh/id_rsa -J durbek_gafurov_0000000854@bastion-1.fabric-testbed.net","./connect.sh")
        print(f"   SSH Command       : {ssh}")
        # print(f"   Components        :  ")
        # for component in node.get_components():
        #     print(f"      Name             : {component.get_name()}")
        #     print(f"      Details          : {component.get_details()}")
        #     print(f"      Disk (G)         : {component.get_disk()}")
        #     print(f"      Units            : {component.get_unit()}")
        #     print(f"      PCI Address      : {component.get_pci_addr()}")
        #     print(f"      Model            : {component.get_model()}")
        #     print(f"      Type             : {component.get_type()}") 
        print(f"   Interfaces        :  ", end = " ")
        for interface in node.get_interfaces():
        #     print(f"       Name                : {interface.get_name()}")
        #     print(f"           Bandwidth           : {interface.get_bandwidth()}")
            # print(f"           VLAN                : {interface.get_vlan()}")  
            # print(f"           MAC                 : {interface.get_mac()}") 
            print(interface.get_os_interface(), end =" ")
    # for network in slice.get_l2networks():
    #     print("Network:")
    #     print(f"    Name:            {network.get_name()}")
    # print(f"Interface Map: {slice.get_interface_map()}")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

----
Node:
   Name              : s1
   SSH Command       : ./connect.sh ubuntu@63.239.135.68
   Interfaces        :   ens7 ens9 ens8 ens10 ----
Node:
   Name              : s2
   SSH Command       : ./connect.sh ubuntu@63.239.135.115
   Interfaces        :   ens10 ens9 ens8 ens7 ----
Node:
   Name              : s3
   SSH Command       : ./connect.sh ubuntu@63.239.135.95
   Interfaces        :   ens8 ens7 ----
Node:
   Name              : h1
   SSH Command       : ./connect.sh ubuntu@63.239.135.106
   Interfaces        :   ens7 ----
Node:
   Name              : h11
   SSH Command       : ./connect.sh ubuntu@63.239.135.69
   Interfaces        :   ens7 ----
Node:
   Name              : h2
   SSH Command       : ./connect.sh ubuntu@63.239.135.97
   Interfaces        :   ens7 ----
Node:
   Name              : h22
   SSH Command       : ./connect.sh ubuntu@63.239.135.96
   Interfaces        :   ens7 

## Configure Nodes


In [8]:
config_threads = {}

In [9]:
host_config_script = "sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git clone https://github.com/p4lang/tutorials.git/" 
def addIP(name,net_name,node_addr,node_subnet):
    node = slice.get_node(name=name)        
    h1_os_iface = node.get_interface(network_name=net_name).ip_addr_add(addr=node_addr, subnet=node_subnet)
    config_thread = node.execute_thread(host_config_script)
    config_threads[node]=config_thread
    return node
    
try:
    
    h1 = addIP(h1_name,net_h1_name,h1_addr,h1_subnet)
    h11 = addIP(h11_name,net_h11_name,h11_addr,h1_subnet)
    h2 = addIP(h2_name,net_h2_name,h2_addr,h2_subnet)
    h22 = addIP(h22_name,net_h22_name,h22_addr,h2_subnet)


except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [10]:
def checkIP(node):
    try:        
        stdout, stderr = node.execute(f'ifconfig | grep "inet 10.0."')
        print(stdout)
        print(stderr)
    except Exception as e:
        print(f"Exception: {e}")

In [11]:
try: 
    for node, thread in config_threads.items():
        stdout, stderr = thread.result()
        print(f"Config thread node {node.get_name()} complete")
        checkIP(node)
        # print(f"stdout: {stdout}")
        # print(f"stderr: {stderr}")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 
config_threads = {}

Config thread node h1 complete
        inet 10.0.1.1  netmask 255.255.255.0  broadcast 0.0.0.0


Config thread node h11 complete
        inet 10.0.1.11  netmask 255.255.255.0  broadcast 0.0.0.0


Config thread node h2 complete
        inet 10.0.2.2  netmask 255.255.255.0  broadcast 0.0.0.0


Config thread node h22 complete
        inet 10.0.2.22  netmask 255.255.255.0  broadcast 0.0.0.0




## Configure Switches

Use ssh to configure the ifaces on the switches. This step requires testing the interfaces to figure out which interface is connected to which network.


#### Setup P4 Docker



Below are commands to let sudo work with the global proxy.

In [12]:
def configureSwitch(name,network_names):
    try:
        node = slice.get_node(name=name)
        os_ifaces = ""
        for net_name in network_names:
            iface = node.get_interface(network_name=net_name).get_os_interface()
            if iface is None:
                print("iface is none")
                return
            os_ifaces=os_ifaces+iface+" "

        file_attributes = node.upload_file('./scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
        command=f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh {os_ifaces}  > /tmp/script.log 2>&1' && tail /tmp/script.log"
        config_threads[node] = node.execute_thread(command)
        print(command)


    except Exception as e:
        print(f"Error: {e}")
        traceback.print_exc()

In [13]:
configureSwitch(s1_name,[net_h1_name,net_h11_name,net_s1_s2_name,net_s1_s3_name])
configureSwitch(s2_name,[net_h2_name,net_h22_name,net_s1_s2_name,net_s2_s3_name])
configureSwitch(s3_name,[net_s1_s3_name,net_s2_s3_name])

chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh ens7 ens8 ens10 ens9   > /tmp/script.log 2>&1' && tail /tmp/script.log
chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh ens8 ens10 ens9 ens7   > /tmp/script.log 2>&1' && tail /tmp/script.log
chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh ens8 ens7   > /tmp/script.log 2>&1' && tail /tmp/script.log


In [14]:
try: 
    for node, thread in config_threads.items():
        stdout, stderr = thread.result()
        print(f"Config thread node {node.get_name()} complete")
        print(f"stdout: {stdout}")
        print(f"stderr: {stderr}")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 


Config thread node s1 complete
stdout: V1Switch(
MyParser(),
MyVerifyChecksum(),
MyIngress(),
MyEgress(),
MyComputeChecksum(),
MyDeparser()
) main;
simple_switch --interface 1@ens7 --interface 2@ens8 --interface 3@ens10 --interface 4@ens9 /root/tutorials/exercises/basic_tunnel/basic_tunnel.json
done!

stderr: 
Config thread node s2 complete
stdout: V1Switch(
MyParser(),
MyVerifyChecksum(),
MyIngress(),
MyEgress(),
MyComputeChecksum(),
MyDeparser()
) main;
simple_switch --interface 1@ens8 --interface 2@ens10 --interface 3@ens9 --interface 4@ens7 /root/tutorials/exercises/basic_tunnel/basic_tunnel.json
done!

stderr: 
Config thread node s3 complete
stdout: V1Switch(
MyParser(),
MyVerifyChecksum(),
MyIngress(),
MyEgress(),
MyComputeChecksum(),
MyDeparser()
) main;
simple_switch --interface 1@ens8 --interface 2@ens7 /root/tutorials/exercises/basic_tunnel/basic_tunnel.json
done!

stderr: 


### Confgure P4 Switch Tables

Edit sX_commands.txt to change the values

In [24]:
h1Mac = h1.get_interface(network_name=net_h1_name).get_mac()
h11Mac = h11.get_interface(network_name=net_h11_name).get_mac()
h2Mac = h2.get_interface(network_name=net_h2_name).get_mac()
h22Mac = h22.get_interface(network_name=net_h22_name).get_mac()
print(h1Mac,h11Mac,h2Mac,h22Mac)

52:85:BD:B7:BC:4D 56:C7:92:93:3D:25 5A:03:7D:DD:45:DD 5A:10:EE:11:85:DC


In [15]:
for switch_name in [s1_name, s2_name, s3_name]:
    switch_node = slice.get_node(name=switch_name)
    management_ip_switch = str(switch_node.get_management_ip())
    print("Swtitch Name        : {}".format(switch_node.get_name()))
    print("Management IP    : {}".format(management_ip_switch))
    
    #Configure P4 Tables
    cmd_file=f'{switch_name}_commands.txt'
    print(cmd_file)
    file_attributes = switch_node.upload_file(f'scripts/{cmd_file}',cmd_file)
    print("file_attributes: {}".format(file_attributes))

    stdout = switch_node.execute(f"sudo sh -c 'cat {cmd_file} | docker exec -i fabric_p4 simple_switch_CLI'")
    print("stdout: {}".format(stdout))

Swtitch Name        : s1
Management IP    : 63.239.135.68
s1_commands.txt
file_attributes: -rw-rw-r--   1 1000     1000          235 02 Sep 00:18 ?
stdout: ('Obtaining JSON from switch...\nDone\nControl utility for runtime P4 table manipulation\nRuntimeCmd: Setting default action of myTunnel_exact\naction:              drop\nruntime data:        \nRuntimeCmd: Adding entry to exact match table myTunnel_exact\nmatch key:           EXACT-00:02\naction:              myTunnel_forward\nruntime data:        00:03\nEntry has been added with handle 0\nRuntimeCmd: Adding entry to exact match table myTunnel_exact\nmatch key:           EXACT-00:16\naction:              myTunnel_forward\nruntime data:        00:04\nEntry has been added with handle 1\nRuntimeCmd: Adding entry to exact match table myTunnel_exact\nmatch key:           EXACT-00:01\naction:              myTunnel_forward\nruntime data:        00:01\nEntry has been added with handle 2\nRuntimeCmd: Adding entry to exact match table myTunne

## The switches are now configured and running. Now we are going to send packets over the switches.

We are going to use `send.py` and `receive.py`. We are going to re-upload them to the servers and use them. Make sure to modify the interface names in the script accordingly.

In [16]:
h1.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h1.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')
h2.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h2.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')
h11.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h11.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')
h22.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h22.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')

SCP upload fail. Slice: P4_VCC_hm, Node: h2, trying again
Fail: ChannelException(2, 'Connect failed')


<SFTPAttributes: [ size=1137 uid=1000 gid=1000 mode=0o100664 atime=1662078075 mtime=1662078075 ]>

In [17]:
print(h1)

-----------------  ----------------------------------------------------------------------------------------------------------------------------------
ID                 4045f660-0ee0-4864-a2a1-7a256c2a46d7
Name               h1
Cores              2
RAM                8
Disk               100
Image              default_ubuntu_20
Image Type         qcow2
Host               max-w5.fabric-testbed.net
Site               MAX
Management IP      63.239.135.106
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/.ssh/id_rsa -J durbek_gafurov_0000000854@bastion-1.fabric-testbed.net ubuntu@63.239.135.106
-----------------  ----------------------------------------------------------------------------------------------------------------------------------


## H1 sends packet, H2 recieves

In [18]:
h1.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py --dst_id 2 10.0.2.2 "message100"\'')
print(h2.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

sniffing on ens7
got a packet
###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = 52:85:bd:b7:bc:4d
  type      = 0x1212
###[ MyTunnel ]### 
     pid       = 2048
     dst_id    = 2
###[ IP ]### 
        version   = 4
        ihl       = 5
        tos       = 0x0
        len       = 30
        id        = 1
        flags     = 
        frag      = 0
        ttl       = 64
        proto     = hopopt
        chksum    = 0x5f4a
        src       = 10.20.5.128
        dst       = 10.0.2.2
        \options   \
###[ Raw ]### 
           load      = 'message100'
###[ Padding ]### 
              load      = '\x00\x00\x00\x00\x00\x00\x00\x00'

got a packet
###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = 52:85:bd:b7:bc:4d
  type      = 0x1212
###[ MyTunnel ]### 
     pid       = 2048
     dst_id    = 2
###[ IP ]### 
        version   = 4
        ihl       = 5
        tos       = 0x0
        len       = 30
        id        = 1
        flags     = 
        frag    

## H11 sends packet, H22 recieves

In [19]:
h11.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py --dst_id 22 10.10.2.2 "message100"\'')
print(h22.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

sniffing on ens7
got a packet
###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = 56:c7:92:93:3d:25
  type      = 0x1212
###[ MyTunnel ]### 
     pid       = 2048
     dst_id    = 22
###[ IP ]### 
        version   = 4
        ihl       = 5
        tos       = 0x0
        len       = 30
        id        = 1
        flags     = 
        frag      = 0
        ttl       = 64
        proto     = hopopt
        chksum    = 0x5ff9
        src       = 10.20.4.199
        dst       = 10.10.2.2
        \options   \
###[ Raw ]### 
           load      = 'message100'
###[ Padding ]### 
              load      = '\x00\x00\x00\x00\x00\x00\x00\x00'

got a packet
###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = 56:c7:92:93:3d:25
  type      = 0x1212
###[ MyTunnel ]### 
     pid       = 2048
     dst_id    = 22
###[ IP ]### 
        version   = 4
        ihl       = 5
        tos       = 0x0
        len       = 30
        id        = 1
        flags     = 
        frag 

## H2 sends packet, H1 recieves

In [20]:
h2.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py --dst_id 1 10.0.1.1 "message100"\'')
print(h1.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

sniffing on ens7
got a packet
###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = 5a:03:7d:dd:45:dd
  type      = 0x1212
###[ MyTunnel ]### 
     pid       = 2048
     dst_id    = 1
###[ IP ]### 
        version   = 4
        ihl       = 5
        tos       = 0x0
        len       = 30
        id        = 1
        flags     = 
        frag      = 0
        ttl       = 64
        proto     = hopopt
        chksum    = 0x6088
        src       = 10.20.5.67
        dst       = 10.0.1.1
        \options   \
###[ Raw ]### 
           load      = 'message100'
###[ Padding ]### 
              load      = '\x00\x00\x00\x00\x00\x00\x00\x00'

got a packet
###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = 5a:03:7d:dd:45:dd
  type      = 0x1212
###[ MyTunnel ]### 
     pid       = 2048
     dst_id    = 1
###[ IP ]### 
        version   = 4
        ihl       = 5
        tos       = 0x0
        len       = 30
        id        = 1
        flags     = 
        frag     

## H22 sends packet, H11 recieves

In [21]:
h22.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py --dst_id 1 10.0.1.1 "message100"\'')
print(h1.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

sniffing on ens7
got a packet
###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = 5a:10:ee:11:85:dc
  type      = 0x1212
###[ MyTunnel ]### 
     pid       = 2048
     dst_id    = 1
###[ IP ]### 
        version   = 4
        ihl       = 5
        tos       = 0x0
        len       = 30
        id        = 1
        flags     = 
        frag      = 0
        ttl       = 64
        proto     = hopopt
        chksum    = 0x60da
        src       = 10.20.4.241
        dst       = 10.0.1.1
        \options   \
###[ Raw ]### 
           load      = 'message100'
###[ Padding ]### 
              load      = '\x00\x00\x00\x00\x00\x00\x00\x00'

got a packet
###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = 5a:10:ee:11:85:dc
  type      = 0x1212
###[ MyTunnel ]### 
     pid       = 2048
     dst_id    = 1
###[ IP ]### 
        version   = 4
        ihl       = 5
        tos       = 0x0
        len       = 30
        id        = 1
        flags     = 
        frag    

## Delete Slice

In [22]:
# try:
#     slice = fablib.get_slice(name=slice_name)
#     slice.delete()
# except Exception as e:
#     print(f"Fail: {e}")
#     traceback.print_exc()